## 基于kashgari的命名实体识别

### 安装

前置准备
gensim--3.8.1  
tensorflow--2.2.0           
tensorflow-addons--0.9.1  
scikit-learn--0.21.1 
python-- 3.7.1   
pandas--1.2.0 

Kashgari 2.x 安装
通过 github 压缩包下载安装 https://github.com/BrikerMan/Kashgari 

### 准备工作

加载人民日报标注数据

In [1]:
from kashgari.corpus import ChineseDailyNerCorpus

In [2]:
train_x, train_y = ChineseDailyNerCorpus.load_data('train')
valid_x, valid_y = ChineseDailyNerCorpus.load_data('valid')
test_x, test_y = ChineseDailyNerCorpus.load_data('test')

2021-01-19 10:38:02,586 [DEBUG] kashgari - loaded 20864 samples from C:\Users\DELL\.kashgari\datasets\china-people-daily-ner-corpus\example.train. Sample:
x[0]: ['“', '文', '革', '”', '中', '《', '战', '争', '和', '人', '》', '一', '百', '二', '十', '万', '字', '的', '第', '一', '稿', '被', '毁', '，', '文', '革', '后', '五', '十', '五', '岁', '时', '提', '笔', '重', '写', '，', '创', '作', '第', '二', '部', '过', '程', '中', '患', '严', '重', '脑', '震', '荡', '，', '一', '只', '眼', '睛', '失', '明', '…', '…', '这', '一', '切', '仅', '只', '是', '创', '作', '过', '程', '中', '能', '说', '得', '出', '“', '名', '目', '”', '的', '代', '价', '和', '付', '出', '。']
y[0]: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [3]:
print(len(train_x),len(valid_x),len(test_x))

20864 2318 4636


In [11]:
train_x[6],train_y[6]

(['最',
  '终',
  '，',
  '全',
  '国',
  '2',
  '3',
  '个',
  '选',
  '区',
  '将',
  '产',
  '生',
  '1',
  '2',
  '2',
  '名',
  '国',
  '民',
  '议',
  '会',
  '议',
  '员',
  '。'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-ORG',
  'I-ORG',
  'I-ORG',
  'I-ORG',
  'O',
  'O',
  'O'])

多种词向量选择：

- BertEmbedding
- WordEmbedding 
- GPT2Embedding

In [12]:
from kashgari.embeddings import BertEmbedding

bert_embed = BertEmbedding('chinese_L-12_H-768_A-12')

2021-01-19 10:42:06,626 [DEBUG] kashgari - ------------------------------------------------
2021-01-19 10:42:06,627 [DEBUG] kashgari - Loaded transformer model's vocab
2021-01-19 10:42:06,627 [DEBUG] kashgari - config_path       : chinese_L-12_H-768_A-12\bert_config.json
2021-01-19 10:42:06,629 [DEBUG] kashgari - vocab_path      : chinese_L-12_H-768_A-12\vocab.txt
2021-01-19 10:42:06,629 [DEBUG] kashgari - checkpoint_path : chinese_L-12_H-768_A-12\bert_model.ckpt
2021-01-19 10:42:06,630 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[unused31]', '[unused32]', '[unused33]', '[unused

可选模型：

- CNN_LSTM_Model
- BiLSTM_Model
- BiGRU_Model
- BiGRU_CRF_Model

In [15]:
from kashgari.tasks.labeling import BiLSTM_CRF_Model

ner_model = BiLSTM_CRF_Model(bert_embed,sequence_length=20)

 ```get_default_hyper_parameters()``` 

```python
hyper = BiLSTM_CRF_Model.default_hyper_parameters()
print(hyper)
#{'layer_blstm': {'units': 128, 'return_sequences': True}, 'layer_dropout': {'rate': 0.4}, 'layer_time_distributed': {}, #'layer_activation': {'activation': 'softmax'}}
hyper["layer_blstm"]["units"] = 100 # 修改参数
ner_model = BiLSTM_CRF_Model(bert_embed,sequence_length=20,hyper_parameters=hyper)
```

模型训练

In [16]:
ner_model.fit(train_x,
          train_y,
          x_validate=valid_x,   
          y_validate=valid_y,
          epochs=5, batch_size=8)

Preparing text vocab dict: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2318/2318 [00:00<00:00, 129117.21it/s]
2021-01-19 10:47:35,728 [DEBUG] kashgari - --- Build vocab dict finished, Total: 3500 ---
2021-01-19 10:47:35,729 [DEBUG] kashgari - Top-10: ['[PAD]', '[UNK]', '[CLS]', '[SEP]', '，', '的', '。', '国', '一', '、']
Preparing text vocab dict: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2318/2318 [00:00<00:00, 165899.88it/s]
2021-01-19 10:47:35,856 [DEBUG] kashgari - --- Build vocab dict finished, Total: 8 ---
2021-01-19 10:47:35,857 [DEBUG] kashgari - Top-10: ['[PAD]', 'O', 'I-ORG', 'I-LOC', 'B-LOC', 'I-PER', 'B-ORG', 'B-PER']
2021-01-19 10:47:40,662 [DEBUG] kashgari - fit input shape: (2, 8, 20)
2021-01-19 10:47:40,663 [DEBUG] kashgari - fit inp

Epoch 1/5
2608/2608 [==============================] - 518s 199ms/step - loss: 2.0128 - accuracy: 0.9627 - val_loss: 16.3738 - val_accuracy: 0.9797
Epoch 2/5
2608/2608 [==============================] - 512s 196ms/step - loss: 0.8071 - accuracy: 0.9775 - val_loss: 11.8641 - val_accuracy: 0.9784
Epoch 3/5
2608/2608 [==============================] - 512s 196ms/step - loss: 0.6166 - accuracy: 0.9794 - val_loss: 9.8969 - val_accuracy: 0.9773
Epoch 4/5
2608/2608 [==============================] - 506s 194ms/step - loss: 0.4978 - accuracy: 0.9822 - val_loss: 9.0293 - val_accuracy: 0.9711
Epoch 5/5
2608/2608 [==============================] - 520s 199ms/step - loss: 0.4395 - accuracy: 0.9834 - val_loss: 8.3559 - val_accuracy: 0.9714


In [ ]:
验证模型，此方法将打印出详细的验证报告：

In [21]:
ner_model.evaluate(test_x, test_y,truncating=True)

2021-01-19 14:12:50,676 [DEBUG] kashgari - predict seq_length: 20, input: (2, 4636, 20)


145/145 [==============================] - 67s 465ms/step


2021-01-19 14:14:01,807 [DEBUG] kashgari - predict output: (4636, 20)
2021-01-19 14:14:01,808 [DEBUG] kashgari - predict output argmax: [[0 1 1 ... 1 1 1]
 [0 1 1 ... 6 2 2]
 [0 1 1 ... 1 1 1]
 ...
 [0 1 1 ... 1 1 1]
 [0 6 2 ... 1 0 1]
 [0 6 2 ... 1 1 1]]



           precision    recall  f1-score   support

      ORG     0.8962    0.8151    0.8537      1271
      LOC     0.9014    0.9087    0.9051      1862
      PER     0.9742    0.9601    0.9671      1103

micro avg     0.9192    0.8940    0.9064      4236
macro avg     0.9188    0.8940    0.9058      4236



{'detail': {'ORG': {'precision': 0.8961937716262975,
   'recall': 0.8151062155782848,
   'f1-score': 0.853728883395138,
   'support': 1271},
  'LOC': {'precision': 0.9014384656366542,
   'recall': 0.9087003222341569,
   'f1-score': 0.9050548274939824,
   'support': 1862},
  'PER': {'precision': 0.9742410303587856,
   'recall': 0.9601087941976428,
   'f1-score': 0.9671232876712328,
   'support': 1103}},
 'precision': 0.9188216627096825,
 'recall': 0.8940037771482531,
 'f1-score': 0.9058164508711958,
 'support': 4236}

模型保存：

In [22]:
ner_model.save('ner.h5')

2021-01-19 14:17:00,648 [INFO] kashgari - model saved to D:\ProgrammeWork\practice\WorkOrderBot_rasa\ner.h5


'D:\\ProgrammeWork\\practice\\WorkOrderBot_rasa\\ner.h5'

h5 文件是层次数据格式第 5 代的版本（Hierarchical Data Format，HDF5），它是用于存储科学数据的一种文件格式和库文件，由美国超级计算与应用中心研发的文件格式，用以存储和组织大规模数据。目前由非营利组织 HDF 小组提供支持。h5 文件将包含：
- 模型的结构，以便重构该模型
- 模型的权重
- 训练配置（损失函数，优化器等）
- 优化器的状态，以便于从上次训练中断的地方开始

对新输入进行预测：

In [26]:
def ner_predict(input_sen,model):
    ners = model.predict([[char for char in input_sen]])
    return ners

In [28]:
import kashgari
loaded_model = kashgari.utils.load_model('ner.h5')
input_sen = "新华社是个不错的单位"
ners = ner_predict(input_sen,loaded_model)
ners

d:\anaconda\envs\kashgari_py3.7\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The 'load_model' function is deprecated, use 'XX_Model.load_model' instead
  
2021-01-19 14:24:03,484 [DEBUG] kashgari - ------------------------------------------------
2021-01-19 14:24:03,484 [DEBUG] kashgari - Loaded transformer model's vocab
2021-01-19 14:24:03,485 [DEBUG] kashgari - config_path       : chinese_L-12_H-768_A-12\bert_config.json
2021-01-19 14:24:03,486 [DEBUG] kashgari - vocab_path      : chinese_L-12_H-768_A-12\vocab.txt
2021-01-19 14:24:03,486 [DEBUG] kashgari - checkpoint_path : chinese_L-12_H-768_A-12\bert_model.ckpt
2021-01-19 14:24:03,486 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[u

1/1 [==============================] - 0s 998us/step


2021-01-19 14:24:12,416 [DEBUG] kashgari - predict output: (1, 12)
2021-01-19 14:24:12,417 [DEBUG] kashgari - predict output argmax: [[0 6 2 2 1 1 1 1 1 1 1 0]]


[['B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]